# 🚀 Fault Detection with GC10-DET

This notebook is the **data preparation stage** of a portfolio project for defect detection on industrial surfaces.  
Dataset: [GC10-DET](https://www.kaggle.com/datasets/alex000kim/gc10det/data)

Steps in this section:
1. Explore the dataset  
2. Visualize samples with bounding boxes  
3. Build a custom PyTorch `Dataset`  
4. Add data augmentation  
5. Create DataLoaders for training/validation


In [ ]:
# If running on Kaggle, most are preinstalled
!pip install albumentations==1.4.0 --quiet

import os
import glob
import xml.etree.ElementTree as ET
import random
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2


## 🗂 Dataset Overview

- **GC10-DET** contains 10 defect classes of metal surfaces.  
- Images are stored under `images/` and annotations under `annotations/` (Pascal VOC XML).  
- Task: object detection (bounding boxes + class).

We'll parse the annotations to understand:
- How many images  
- Class distribution  
- Example annotation structure


In [ ]:
source_folders = []
for folder in os.listdir('/kaggle/input/gc10det'):
    try:
        int(folder)
        source_folders.append(os.path.join('/kaggle/input/gc10det', folder))
    except:
        pass

In [ ]:
source_folders

In [ ]:
import os
import shutil
import glob

def combine_images_from_folders(source_folders, destination_folder):
    """
    Combines all image files from a list of source folders into a single
    destination folder.

    Args:
        source_folders (list): A list of strings, where each string is the
                               path to a folder containing images.
        destination_folder (str): The path to the folder where all images will
                                  be copied. This folder will be created if it
                                  does not exist.
    """
    # Create the destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)
    
    # Common image file extensions
    image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff')
    
    total_files_copied = 0

    print(f"Starting to combine images into '{destination_folder}'...")

    for folder in source_folders:
        if not os.path.isdir(folder):
            print(f"Warning: Source folder '{folder}' not found. Skipping.")
            continue
            
        print(f"Processing folder: '{folder}'")
        
        # Use os.walk to find files in the current folder and its subdirectories
        for root, _, files in os.walk(folder):
            for filename in files:
                # Check if the file has a valid image extension
                if filename.lower().endswith(image_extensions):
                    source_path = os.path.join(root, filename)
                    dest_path = os.path.join(destination_folder, filename)
                    
                    # Handle potential duplicate filenames by adding a number
                    if os.path.exists(dest_path):
                        base, ext = os.path.splitext(filename)
                        counter = 1
                        while os.path.exists(os.path.join(destination_folder, f"{base}_{counter}{ext}")):
                            counter += 1
                        dest_path = os.path.join(destination_folder, f"{base}_{counter}{ext}")

                    try:
                        # shutil.copy2 preserves file metadata (timestamps, etc.)
                        shutil.copy2(source_path, dest_path)
                        total_files_copied += 1
                    except IOError as e:
                        print(f"  Error copying file '{source_path}': {e}")
                    
    print("\n--- Combination Complete ---")
    print(f"Total files copied: {total_files_copied}")



In [ ]:
images_source = "Images_all"
combine_images_from_folders(source_folders, images_source)

In [ ]:
# mapping from the label-with-number to clean English name
CLASS_MAP = {
    "1_chongkong": "punching_hole",
    "2_hanfeng": "welding_line",
    "3_yueyawan": "crescent_gap",
    "4_shuibian": "water_spot",
    "5_youbian": "oil_spot",
    "6_siban": "silk_spot",
    "7_yiwu": "inclusion",
    "8_xiahen": "rolled_pit",
    "9_zhehen": "crease",
    "10_yaozhe": "waist_folding"
}


In [ ]:
IMG_DIR = "/kaggle/working/Images_all"
ANN_DIR = "/kaggle/input/gc10det/lable"
print()
xml_files = glob.glob(os.path.join(ANN_DIR, "*.xml"))
print(f"Total annotations: {len(xml_files)}")

# Parse one file to inspect structure
tree = ET.parse(xml_files[0])
root = tree.getroot()

for obj in root.findall("object"):
    name = obj.find("name").text
    bbox = obj.find("bndbox")
    coords = [int(bbox.find(tag).text) for tag in ["xmin","ymin","xmax","ymax"]]
    print(name, coords)


## 🖼 Sample Images

Let's plot a few raw images with their bounding boxes to verify labels.


In [ ]:
def plot_image_with_boxes(img_path, ann_path):
    img = np.array(Image.open(img_path).convert("RGB"))
    tree = ET.parse(ann_path)
    boxes = []
    labels = []
    for obj in tree.findall("object"):
        labels.append(obj.find("name").text)
        bb = obj.find("bndbox")
        boxes.append([int(bb.find(t).text) for t in ["xmin","ymin","xmax","ymax"]])
    plt.imshow(img)
    for box, lbl in zip(boxes, labels):
        x1, y1, x2, y2 = box
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                          fill=False, color='red', linewidth=2))
        plt.text(x1, y1-5, lbl, color='red', fontsize=8, backgroundcolor='white')
    plt.axis("off")

sample_files = random.sample(xml_files, 3)
plt.figure(figsize=(15,5))
for i, ann in enumerate(sample_files):
    plt.subplot(1,3,i+1)
    img_file = os.path.join(IMG_DIR, os.path.basename(ann).replace(".xml",".jpg"))
    plot_image_with_boxes(img_file, ann)
plt.tight_layout()


# 🛠️ Install & Import Ultralytics YOLO

We will use the **Ultralytics YOLO** implementation (v8/v11).  
It is lightweight, easy to train, and perfect for deployment.


In [ ]:
!pip install ultralytics
from ultralytics import YOLO
import os, glob, random, shutil
from pathlib import Path
import xml.etree.ElementTree as ET


### 1️⃣ Convert VOC XML annotations → YOLO format

We create the standard YOLO folder structure:



In [ ]:
import os, glob, shutil
from tqdm import tqdm

# paths
ROOT = "/kaggle/working/gc10_yolo"
IMG_DIR = "/kaggle/working/Images_all"
ANN_DIR = "/kaggle/input/gc10det/lable"

In [ ]:
import os, glob, shutil, random
from xml.etree import ElementTree as ET
from PIL import Image
from tqdm import tqdm


os.makedirs(f"{ROOT}/images/train", exist_ok=True)
os.makedirs(f"{ROOT}/images/val", exist_ok=True)
os.makedirs(f"{ROOT}/labels/train", exist_ok=True)
os.makedirs(f"{ROOT}/labels/val", exist_ok=True)

CLASSES = ["scratch"]   # only one class

def xml_to_yolo(xml_file, txt_file, img_w, img_h):
    tree = ET.parse(xml_file)
    lines = []
    for obj in tree.findall("object"):
        bnd = obj.find("bndbox")
        xmin, ymin = float(bnd.find("xmin").text), float(bnd.find("ymin").text)
        xmax, ymax = float(bnd.find("xmax").text), float(bnd.find("ymax").text)

        x_c = ((xmin + xmax) / 2) / img_w
        y_c = ((ymin + ymax) / 2) / img_h
        w   = (xmax - xmin) / img_w
        h   = (ymax - ymin) / img_h
        lines.append(f"0 {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}")  # always class 0

    with open(txt_file, "w") as f:
        f.write("\n".join(lines))


# split train/val and copy
all_xml = sorted(glob.glob(f"{ANN_DIR}/*.xml"))
random.shuffle(all_xml)
split = int(0.8 * len(all_xml))
train_xml, val_xml = all_xml[:split], all_xml[split:]

for subset, xmls in [("train", train_xml), ("val", val_xml)]:
    for xml in tqdm(xmls, desc=f"Preparing {subset}"):
        img_path = os.path.join(IMG_DIR, os.path.basename(xml).replace(".xml", ".jpg"))
        dst_img = f"{ROOT}/images/{subset}/{os.path.basename(img_path)}"
        dst_txt = f"{ROOT}/labels/{subset}/{os.path.basename(xml).replace('.xml', '.txt')}"
        w, h = Image.open(img_path).size
        shutil.copy(img_path, dst_img)
        xml_to_yolo(xml, dst_txt, w, h)


### 2️⃣ Define YOLO dataset configuration


In [ ]:
import yaml

data_yaml = {
    "path": ROOT,
    "train": "images/train",
    "val": "images/val",
    "nc": len(CLASSES),
    "names": CLASSES,
}

yaml_path = f"{ROOT}/data.yaml"
with open(yaml_path, "w") as f:
    yaml.safe_dump(data_yaml, f, sort_keys=False)

print(open(yaml_path).read())


### 3️⃣ Train YOLOv8 on GC10


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov9c.pt")  # or yolov8n.pt for speed
model.train(
    data=yaml_path,
    epochs=100,
    imgsz=256,
    batch=16,
    project="gc10_yolo",
    name="exp",
    workers=4,
    patience = 20
)

### 4️⃣ Validate & Visualize YOLOv9-e results

We run evaluation on the validation split and plot some detections.


In [ ]:
# Run evaluation on the validation set
metrics = model.val(data=yaml_path, imgsz=640, batch=16)
print(metrics)   # mAP, precision, recall, etc.

# Predict on a few val images & save results
pred_results = model.predict(
    source=f"{ROOT}/images/val",
    conf=0.25,   # confidence threshold
    save=True,   # saves annotated images into runs/predict/
    imgsz=640
)

# Show one of the saved prediction images
import matplotlib.pyplot as plt
import glob

saved_imgs = glob.glob("runs/detect/predict*/**/*.jpg", recursive=True)
plt.figure(figsize=(10,10))
plt.imshow(plt.imread(saved_imgs[0]))
plt.axis("off")
plt.show()


### 5️⃣ Save & Export YOLOv9-e weights

We store the trained model weights and also export to ONNX or TorchScript for deployment.


In [ ]:
# Save the trained model (PyTorch format)
best_path = model.ckpt_path if hasattr(model, "ckpt_path") else model.trainer.best
print(f"Best weights: {best_path}")

# Optionally export to other formats
model.export(format="onnx")        # ONNX
model.export(format="torchscript") # TorchScript


In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(plt.imread(saved_imgs[0]))
plt.axis("off")
plt.show()